In [44]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

file_to_load= "../Project_1/employee_reviews.csv"
review_data= pd.read_csv(file_to_load)


In [45]:
review_data.head(1)

,Unnamed: 0,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link
0,1,google,none,"Dec 11, 2018",Current Employee - Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,none,5.0,4.0,5.0,5.0,4.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...


In [46]:
company_summary_df=review_data.groupby(["company"]).agg({"company":["count"],"overall-ratings":["mean","median"]}).reset_index()
company_summary_df.columns = ["Company" , "#_of_reviews", "avg_rating","median_rating"]
company_summary_df

,Company,#_of_reviews,avg_rating,median_rating
0,amazon,26430,3.587363,4.0
1,apple,12950,3.958224,4.0
2,facebook,1590,4.511950,5.0
3,google,7819,4.339430,5.0
4,microsoft,17930,3.816564,4.0
5,netflix,810,3.411111,4.0


In [47]:
def time_convert(time):
    #if type(time) != 'str':
         #return(time)
    try:
        conv_time = time.replace(",","").strip()
        new_time = datetime.datetime.strptime(conv_time, '%b %d %Y').date()
    except:
        new_time = (np.NaN)
    return(new_time)
    
review_data['dates'] = review_data['dates'].apply(time_convert)

review_data.head()

,Unnamed: 0,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link
0,1,google,none,2018-12-11,Current Employee - Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,none,5.0,4.0,5.0,5.0,4.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...
1,2,google,"Mountain View, CA",2013-06-21,Former Employee - Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,4.0,2.0,3.0,3.0,5.0,3.0,2094,https://www.glassdoor.com/Reviews/Google-Revie...
2,3,google,"New York, NY",2014-05-10,Current Employee - Software Engineer III,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5.0,5.0,4.0,5.0,5.0,4.0,949,https://www.glassdoor.com/Reviews/Google-Revie...
3,4,google,"Mountain View, CA",2015-02-08,Current Employee - Anonymous Employee,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,5.0,2.0,5.0,5.0,4.0,5.0,498,https://www.glassdoor.com/Reviews/Google-Revie...
4,5,google,"Los Angeles, CA",2018-07-19,Former Employee - Software Engineer,"Unique, one of a kind dream job",Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",Promote managers into management for their man...,5.0,5.0,5.0,5.0,5.0,5.0,49,https://www.glassdoor.com/Reviews/Google-Revie...


In [48]:
# Rename columns for merge-ability
review_data=review_data.rename(columns={"dates":"Date"})


In [70]:
amazon_df=(review_data[review_data['company']=='amazon']).dropna()
apple_df=(review_data[review_data['company']=='apple']).dropna()
facebook_df=(review_data[review_data['company']=='facebook']).dropna()
google_df=(review_data[review_data['company']=='google']).dropna()
microsoft_df=(review_data[review_data['company']=='microsoft']).dropna()
netflix_df=(review_data[review_data['company']=='netflix']).dropna()


,Unnamed: 0,company,location,Date,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link
36649,36650,apple,none,2018-12-11,Current Employee - Anonymous Employee,GIS & Data Analyst,"Fun work environment, great managers","Sometimes there's a high volume of work, but s...",none,5.0,none,none,none,none,none,0,https://www.glassdoor.com/Reviews/Apple-Review...
36650,36651,apple,none,2014-08-28,Current Employee - Anonymous Employee,"Challenging, Rewarding, but ZERO work/life bal...","We work with geniuses - in every department, W...",ZERO ZERO ZERO work/life balance. Execs have b...,none,4.0,1.0,3.0,3.0,3.0,3.0,822,https://www.glassdoor.com/Reviews/Apple-Review...
36651,36652,apple,"Tempe, AZ",2017-03-10,Current Employee - Specialist,A Company as Meticulous as Their Products!,"Competitive Pay, Great Benefits, Amazing Peopl...",None! I had one of the best times ever at this...,You guys are awesome. Keep up the good work!,5.0,4.0,5.0,5.0,4.0,5.0,248,https://www.glassdoor.com/Reviews/Apple-Review...
36652,36653,apple,"Lakewood, CO",2015-05-13,Current Employee - Apple At Home Advisor,At-Home-Advisor,The position is at home. The company is AMAZIN...,The schedule changes every 90 days and you hav...,The management consists of a team manager and ...,5.0,3.0,5.0,5.0,4.0,5.0,442,https://www.glassdoor.com/Reviews/Apple-Review...
36653,36654,apple,none,2018-12-08,Current Employee - Anonymous Employee,Good,"Salary, Hours, Time, Career opportunities",They only offer part-time hours,none,5.0,3.0,3.0,2.0,5.0,3.0,1,https://www.glassdoor.com/Reviews/Apple-Review...


In [75]:
amazon_df['Date'] = pd.to_datetime(amazon_df['Date'],format=None)

In [76]:
#Apple stock data
AAPL_file= "../Project_1/worldtradingdata-history-AAPL.csv"
AAPL_df= pd.read_csv(AAPL_file)

#FB stock data
FB_file= "../Project_1/worldtradingdata-history-FB.csv"
FBL_df= pd.read_csv(FB_file)

#Microsoft stock data
MSFT_file= "../Project_1/worldtradingdata-history-MSFT.csv"
MSFT_df= pd.read_csv(MSFT_file)

#Google stock data
GOOG_file= "../Project_1/worldtradingdata-history-GOOG.csv"
GOOG_df= pd.read_csv(GOOG_file)

#Amazon stock data
AMZN_file= "../Project_1/worldtradingdata-history-AMZN.csv"
AMZN_df= pd.read_csv(AMZN_file)
AMZN_df.head()


,Date,Open,Close,High,Low,Volume
0,1997-05-15,1.96,1.96,2.50,1.93,6006000
1,1997-05-16,1.97,1.73,1.98,1.71,1223800
2,1997-05-19,1.76,1.71,1.77,1.62,508900
3,1997-05-20,1.73,1.64,1.75,1.64,455600
4,1997-05-21,1.63,1.43,1.65,1.37,1571100


In [77]:
type(AMZN_df['Date'][0])

str

In [78]:
AMZN_df['Date'] = pd.to_datetime(AMZN_df['Date'],format=None)

In [80]:
Amazon_AMZN=pd.merge(amazon_df, AMZN_df, how='left', on ='Date')
Amazon_AMZN.head()

,Unnamed: 0,company,location,Date,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,...,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link,Open,Close,High,Low,Volume
0,7821,amazon,"Sunnyvale, CA",2018-12-10,"Current Employee - Manager, Software Developme...",Software Development Engineering Manager,"Work hard, have fun, make history. Be proud of...",RUS grand each year related to last year's sto...,none,5.0,...,5.0,4.0,5.0,0,https://www.glassdoor.com/Reviews/Amazon-Revie...,1623.84,1641.03,1657.99,1590.87,7494808.0
1,7822,amazon,"Seattle, WA",2016-01-10,Current Employee - Anonymous Employee,You Get What You Put In,"Really smart people, a lot of opportunity for ...",You have to be self motivated. NO ONE will hol...,More on-boarding training before new employees...,5.0,...,5.0,5.0,5.0,1759,https://www.glassdoor.com/Reviews/Amazon-Revie...,NaN,NaN,NaN,NaN,NaN
2,7823,amazon,"Seattle, WA",2016-02-20,Current Employee - Senior Engineering Manager,"Exciting Work, Abusive Culture","Jeff Bezos and his ""S-Team"" are brilliant and ...","The management process is abusive, and I'm cur...",Don't pretend that the recent NY Times article...,3.0,...,3.0,4.0,4.0,1404,https://www.glassdoor.com/Reviews/Amazon-Revie...,NaN,NaN,NaN,NaN,NaN
3,7824,amazon,"Seattle, WA",2013-12-17,Current Employee - Software Development Manager,"Can be amazing for some people, horrible for o...",Amazon is doing lot's of cool stuff...but lots...,- You're responsible for your own career progr...,Stack ranking is a horrible practice since it'...,4.0,...,5.0,5.0,4.0,1275,https://www.glassdoor.com/Reviews/Amazon-Revie...,390.65,387.65,391.36,386.50,2348377.0
4,7825,amazon,"Seattle, WA",2017-03-23,Former Employee - Anonymous Employee,We are now in a world where we are condescende...,This company gets A list performance from C li...,Maybe because the company recruits C listers w...,Bezos you bought the Washington Post and hired...,4.0,...,1.0,5.0,1.0,409,https://www.glassdoor.com/Reviews/Amazon-Revie...,848.20,847.38,850.89,844.80,1952964.0
